In [1]:
import torch
from torch.utils import data

In [2]:
class Dataset(data.Dataset):
    def __init__(self, list_IDs):
        self.list_IDs = list_IDs
    
    def __len__(self):
        return len(self.list_IDs)
    
    def __getitem__(self, index):
        ID = self.list_IDs[index]
        X = torch.load('./data/' + ID + '.pt')
        return X

### Set up PyTorch

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [4]:
# Parameters
params = {'batch_size': 64,
         'shuffle': True,
         'num_workers': 4}

In [5]:
# Max epochs
max_epochs = 10

In [ ]:
# Datasets
partition 